https://huggingface.co/

# Før træning:
*prompt = "because he runs about making mischief, and telling tales"*

because he runs about making mischief, and telling tales of his own adventures.
The story is told in the form that a young man named John (played by actor Michael Caine)
has been kidnapped from an unknown location on Earth where there are no humans to protect
him or anyone else who might be involved. He's taken with some otherworldly beings called
"the Guardians" but they're not real people so it seems like this guy isn't really
interested at all when things go wrong for them as

# Efter træning:
*prompt = "because he runs about making mischief, and telling tales"*

because he runs about making mischief, and telling tales of the
great beasts that hunt him.” He was a man who had never been to any place where there were no elephants or wolves in his line; but when Mowgli came down from camp with all four seals at their feet (and they did not know what it meant), Bagheera told them: “I am going out into this world for my own sake--for I have seen many things which are very foolish


In [ ]:
#!wget https://raw.githubusercontent.com/jakupwhansen/Public_Files/main/the_jungle_book.txt -O the_jungle_book.txt

--2024-04-11 11:12:54--  https://raw.githubusercontent.com/jakupwhansen/Public_Files/main/the_jungle_book.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 304371 (297K) [text/plain]
Saving to: ‘the_jungle_book.txt’

the_jungle_book.txt 100%[===================>] 297.24K  --.-KB/s    in 0.02s   

2024-04-11 11:12:54 (12.8 MB/s) - ‘the_jungle_book.txt’ saved [304371/304371]



In [17]:
with open("txt/odessy.txt", "r", encoding="utf-8") as file:
    text = file.read()

# Simple preprocessing: split the text into chunks if needed
# For demonstration, let's just use the full text as is


In [18]:
# Ensure necessary libraries are installed and updated
!pip install accelerate -U
!pip install transformers datasets -U

# Your code for loading the dataset, initializing the model, tokenizer, and trainer follows here


In [32]:
!pip install "accelerate>=0.21.0" -U
!pip install "transformers[torch]" -U

In [33]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch

# Assuming you're using a GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Ensure the tokenizer recognizes a pad token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained('gpt2')
model.config.pad_token_id = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)

# Example preprocessing of your text
# Let's assume 'text' is a string containing the content of "The Jungle Book"

text_file = 'txt/odessy.txt' # You need to download the book text and specify its local path here

# Create a dataset and a data collator for language modeling
dataset = TextDataset(
    tokenizer=tokenizer,
    file_path=text_file,
    block_size=128  # Adjust based on your model and preferences
)

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False  # For causal (not masked) language modeling
)

# Training arguments
training_args = TrainingArguments(
    output_dir='./results',          # Where to store the training outputs
    overwrite_output_dir=True,
    num_train_epochs=3,              # Number of epochs (adjust as needed)
    per_device_train_batch_size=4,   # Batch size (adjust based on your GPU)
    save_steps=10_000,               # Save the model every 10,000 steps
    save_total_limit=2,              # Only keep the 2 most recent checkpoints
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Start training. Hvis den lukkes, så bruges pre-trained
# modellen, og det fungerer fint
trainer.train()

# Save the trained model
model.save_pretrained('./your_trained_model')
tokenizer.save_pretrained('./your_trained_model')


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.21.0`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

## **Så er modellen trænet med min egen text. Nu skal vi bruge modellen**

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Jeg har gemt min trænet model i drive under /public/your_trained_model/
model_path = './your_trained_model'  # Update this to your model's directory

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)


In [ ]:
instruction = "You are a Large Language Model named ArthurAI that has been trained to generate text based on the given prompt. If you are asked a question, you will only provide the answer to the question. If you are asked to complete a sentence, you will only provide the completion. If you are asked to generate text, you will generate text based on the given prompt. You are asked: "
prompt = "The mysterious forest"  # Example 1 prompt
prompt = "because he runs about making mischief, and telling tales"  # Example 2 prompt



In [ ]:
input_ids = tokenizer.encode(instruction + prompt, return_tensors='pt')

In [ ]:
# Generate text
output_sequences = model.generate(
    input_ids=input_ids,
    max_length=100,  # Maximum length of the generated text
    num_return_sequences=1,  # Number of generated sequences
    temperature=0.7,  # Sampling temperature
    top_k=50,  # Only consider the top k tokens at each step
    top_p=0.9,  # Nucleus sampling
    repetition_penalty=1.2,  # Penalty for repetition
)

# Decode the generated text
generated_text = tokenizer.decode(output_sequences[0], skip_special_tokens=True)
print(generated_text)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


because he runs about making mischief, and telling tales of the
great beasts that hunt him.” He was a man who had never been to any place where there were no elephants or wolves in his line; but when Mowgli came down from camp with all four seals at their feet (and they did not know what it meant), Bagheera told them: “I am going out into this world for my own sake--for I have seen many things which are very foolish
